In [ ]:
%run init_notebook.py

In [ ]:
import pandas as pd
import numpy as np

import os
import datetime as dt
from functools import wraps, reduce

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from settings import RAW_DATA_DIR, WORK_DATA_DIR
from utils.utils import *
from utils.load import save_file, load_data
from utils.cast_data import apply_datetime_format, apply_textmonth_to_nummonth

In [4]:
# load raw format
fears = load_data(file_name="fears.csv", file_path=RAW_DATA_DIR)

cbeo_vix = load_data(file_name="cbeo_vix.csv", file_path=RAW_DATA_DIR).rename(columns={"Date": "date"})
cbeo_vix.drop_duplicates("date", keep="last", inplace=True)
cbeo_vix.reset_index(drop=True, inplace=True)

naaim_risk = load_data(file_name="naaim_risk.xlsx", file_path=RAW_DATA_DIR).rename(columns={"Date": "date",
                                                                                  "NAAIM Number": "naaim_ind",
                                                                                 "Standard Deviation": "naaim_std"}).iloc[:-4].sort_values("date").reset_index(drop=True)

ff_factors5 = load_data(file_name="ff_factors5.csv", file_path=RAW_DATA_DIR, skiprows=3).add_prefix("ff_").rename(columns={"ff_Unnamed: 0" : "date",
                                                                                                                           "ff_Mkt-RF": "ff_M_RF"}).iloc[:-1]
ff_factors3 = load_data(file_name="ff_factors3.csv", file_path=RAW_DATA_DIR, skiprows=3).add_prefix("ff_").add_suffix("_3").rename(columns={"ff_Unnamed: 0_3" : "date",
                                                                                                                                            "ff_Mkt-RF_3": "ff_M_RF_3"}).iloc[:-1]

us_termspread = load_data(file_name="us_termspread.csv", file_path=RAW_DATA_DIR).rename(columns={"DATE": "date", "TEDRATE": "termspread"})
us_termspread["termspread"] = us_termspread["termspread"].replace({"." : np.nan})

aaii_bull = load_data(file_name="aaii_bull.xlsx", file_path=RAW_DATA_DIR, sheet_name="raw_data").iloc[:, :10].add_prefix("aaii_").rename(columns={"aaii_Reported Date": "date"})
sp500 = load_data(file_name="sp500_yf.xlsx", file_path=RAW_DATA_DIR, sheet_name="sp500").add_prefix("sp_").rename(columns=
                                                                                                                  {
                                                                                                                      "sp_Date": "date",
                                                                                                                      "sp_Volume": "sp_volume",
                                                                                                                      "sp_Close*": "sp_close",
                                                                                                                      "sp_Adj Close**": "sp_adj_close"
                                                                                                                  }
                                                                                                                 ).sort_values("date", ascending=True).reset_index(drop=True)

pc1 = load_data(file_name="cboe_putcall_total_2003-2012.csv", file_path=RAW_DATA_DIR, skiprows=2).rename(columns={"Trade_date": "date", "P/C Ratio": "pc_ratio"})
pc2 = load_data(file_name="cboe_putcall_total_2006-2019.csv", file_path=RAW_DATA_DIR, skiprows=2).rename(columns={"DATE": "date",
                                                                                                        "CALLS": "Call",
                                                                                                        "PUTS": "Put",
                                                                                                        "TOTAL": "Total",
                                                                                                        "P/C Ratio": "pc_ratio"})
goog_sent = load_data(file_name="gog_sent.csv", file_path=RAW_DATA_DIR)
goog_sent = goog_sent[goog_sent.country=="United States"].drop(["Unnamed: 0", "country"], axis=1).rename(columns={"sentiment_combined": "goog_sent",
                                                                                                                  "week": "date"}).reset_index(drop=True)
;

Unknown columns found, columns were not casted
['Date']
Exceptions were found
[KeyError('Date')]
Unknown columns found, columns were not casted
['Date', 'Standard Deviation', 'NAAIM Number']
Exceptions were found
[KeyError('Date'), KeyError('Standard Deviation'), KeyError('NAAIM Number')]
Unknown columns found, columns were not casted
['Unnamed: 0', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
Exceptions were found
[KeyError('Unnamed: 0'), KeyError('Mkt-RF'), KeyError('SMB'), KeyError('HML'), KeyError('RMW'), KeyError('CMA'), KeyError('RF')]
Unknown columns found, columns were not casted
['Unnamed: 0', 'Mkt-RF', 'SMB', 'HML', 'RF']
Exceptions were found
[KeyError('Unnamed: 0'), KeyError('Mkt-RF'), KeyError('SMB'), KeyError('HML'), KeyError('RF')]
Unknown columns found, columns were not casted
['DATE', 'TEDRATE']
Exceptions were found
[KeyError('DATE'), KeyError('TEDRATE')]
Unknown columns found, columns were not casted
['Reported Date', 'Bullish', 'Neutral', 'Bearish', 'Bullish 8-week M

''

In [5]:
# create cast_dict
_ = list(fears.columns)
_.extend(list(cbeo_vix.columns))
_.extend(list(naaim_risk.columns))
_.extend(list(ff_factors3.columns))
_.extend(list(ff_factors5.columns))
_.extend(list(us_termspread.columns))
_.extend(list(pc1.columns))
_.extend(list(aaii_bull.columns))
_.extend(list(sp500.columns))
_.extend(list(goog_sent.columns))

_ = list(set(_))
_.sort()
# {item: float for item in _}

In [6]:
# save files in feather format
save_file(fears, file_name="fears.csv", file_path=WORK_DATA_DIR, index=False)
save_file(cbeo_vix, file_name="cbeo_vix.csv", file_path=WORK_DATA_DIR, index=False)
save_file(naaim_risk, file_name="naaim_risk.csv", file_path=WORK_DATA_DIR, index=False)
save_file(ff_factors5, file_name="ff_factors5.csv", file_path=WORK_DATA_DIR, index=False)
save_file(ff_factors3, file_name="ff_factors3.csv", file_path=WORK_DATA_DIR, index=False)
save_file(us_termspread, file_name="us_termspread.csv", file_path=WORK_DATA_DIR, index=False)
save_file(pc1, file_name="put_call1.csv", file_path=WORK_DATA_DIR, index=False)
save_file(pc2, file_name="put_call2.csv", file_path=WORK_DATA_DIR, index=False)
save_file(aaii_bull, file_name="aaii_bull.csv", file_path=WORK_DATA_DIR, index=False)
save_file(sp500, file_name="sp500.csv", file_path=WORK_DATA_DIR, index=False)
save_file(goog_sent, file_name="goog_sent.csv", file_path=WORK_DATA_DIR, index=False)

Load data files

In [18]:
fears = load_data(file_name="fears.csv", file_path=WORK_DATA_DIR)
cbeo_vix = load_data(file_name="cbeo_vix.csv", file_path=WORK_DATA_DIR)
naaim_risk = load_data(file_name="naaim_risk.csv", file_path=WORK_DATA_DIR)
ff_factors5 = load_data(file_name="ff_factors5.csv", file_path=WORK_DATA_DIR)
ff_factors3 = load_data(file_name="ff_factors3.csv", file_path=WORK_DATA_DIR)
us_termspread = load_data(file_name="us_termspread.csv", file_path=WORK_DATA_DIR)
put_call1 = load_data(file_name="put_call1.csv", file_path=WORK_DATA_DIR)
put_call2 = load_data(file_name="put_call2.csv", file_path=WORK_DATA_DIR)
aaii_bull = load_data(file_name="aaii_bull.csv", file_path=WORK_DATA_DIR)
sp500 = load_data(file_name="sp500.csv", file_path=WORK_DATA_DIR)
goog_sent = load_data(file_name="goog_sent.csv", file_path=WORK_DATA_DIR, europe_time_slash=True)

put_call = pd.concat([put_call1,
                      put_call2.set_index("date").loc[put_call1.iloc[-1].date:].iloc[1:].reset_index()],
                     axis=0).reset_index(drop=True)

Exceptions were found
[AssertionError('Non singular values found')]


In [19]:
data_dict = {}
data_dict.update(
    {
        "data_cols" : 
        {
        "fears": list(fears.drop("date", axis=1).columns),
        "cbeo_vix": list(cbeo_vix.drop("date", axis=1).columns),
        "naaim_risk": list(naaim_risk.drop("date", axis=1).columns),
        "ff_factors5": list(ff_factors5.drop("date", axis=1).columns),
        "termspread": list(us_termspread.drop("date", axis=1).columns),
        "put_call": list(put_call.drop("date", axis=1).columns),
        "aai_bull": list(aaii_bull.drop("date", axis=1).columns),
        "sp500": list(sp500.drop("date", axis=1).columns),
            "goog_sent": list(goog_sent.drop("date", axis=1).columns)
        }
    }
)

dict_df = dict(zip( ["fears", "cbeo_vix", "naaim_risk", "ff_factors3", "ff_factors5", "us_termspread", "put_call", "aaii_bull", "sp500", "goog_sent"],
                   [fears, cbeo_vix, naaim_risk, ff_factors3, ff_factors5, us_termspread, put_call, aaii_bull, sp500, goog_sent]))

Week overview

In [ ]:
def is_day(arr,
           day=None):
    _dict = {"Mon": 0,
             "Tue": 1,
             "Wed": 2,
             "Thu": 3,
             "Fri": 4,
             "Sat": 5,
             "Sun": 6}
    
    if day == None:
        return arr.apply(lambda x: x.weekday())
    else:
        return arr.apply(lambda x: x.weekday() == _dict[day]).rename(f"is_{day}")

In [21]:
get_df_time_overview_report(dict_df)

,min,max,days,weeks,days_perc,week_or_day
fears,2004-07-01,2011-12-30,2738,391,0.690650,1.448677
cbeo_vix,1990-01-02,2021-09-30,11594,1656,0.690271,1.448888
naaim_risk,2006-08-02,2022-02-16,5677,811,0.143209,6.991379
ff_factors3,1926-07-01,2021-12-31,34882,4983,0.720945,1.387124
ff_factors5,1963-07-01,2021-12-30,21367,3052,0.689240,1.450971
us_termspread,1986-01-02,2022-01-21,13168,1881,0.714383,1.399957
put_call,2003-10-17,2019-10-04,5831,833,0.689076,1.451581
aaii_bull,1987-07-23,2022-01-05,12585,1798,0.142789,7.007238
sp500,1990-01-02,2021-12-30,11685,1669,0.690030,1.449392
goog_sent,2004-07-05,2014-12-22,3822,546,0.143119,7.000000


In [22]:
aaii_bull["week"] = aaii_bull.date.apply(lambda x: apply_date_to_week(x))
naaim_risk["week"] = naaim_risk.date.apply(lambda x: apply_date_to_week(x))
goog_sent["week"] = goog_sent.date.apply(lambda x: apply_date_to_week(x))

# drop week duplicates
naaim_risk.drop_duplicates("week", inplace=True)
aaii_bull.drop_duplicates("week", inplace=True)
goog_sent.drop_duplicates("week", inplace=True)

NameError: name 'apply_date_to_week' is not defined

In [24]:
# merge weekly data
data_frames = [aaii_bull.rename(columns={"date": "date_aaii"}),
               goog_sent.rename(columns={"date": "date_goog"}),
               naaim_risk.rename(columns={"date": "date_naaim"})]

df_sub = reduce(lambda  left,right: pd.merge(left,right,on=['week'],
                                            how='outer'), data_frames)

In [25]:
# merge daily and weekly data
df = pd.DataFrame(datetime_range(start=dt.date(1990, 1, 1), end=dt.date(2022, 2, 16)), columns=["date"])

df_list = [
    df,
    sp500,
    cbeo_vix,
    fears,
    ff_factors5,
    us_termspread,
    put_call
]

df = reduce(lambda  left,right: pd.merge(left,
                                                right,
                                                on=['date'],
                                                how='left'),
                   df_list)

df["week"] = df.date.apply(lambda x: apply_date_to_week(x))

df = pd.merge(df,
         df_sub,
         on="week",
         how="outer")

df = df[~df.sp_close.isna()]
df["is_thu"] = is_day(df.date, day="Thu")

In [ ]:
def cut_to_weekly_data(df: pd.DataFrame,
                       filter_col: str):
    
    return df[df[filter_col]==True]

In [27]:
df = shift_var_relative_to_df(df, 
                         shift_var="sp_close",
                         no_lags=[-14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7])
df.dropna(inplace=True)

In [29]:
cut_to_weekly_data(df, "is_thu")

,date,sp_Open,sp_High,sp_Low,sp_close,sp_adj_close,sp_volume,vixo,vixh,vixl,...,sp_close_lead2,sp_close_lead1,sp_close_lead0,sp_close_lag1,sp_close_lag2,sp_close_lag3,sp_close_lag4,sp_close_lag5,sp_close_lag6,sp_close_lag7
6058,2006-08-03,1278.22,1283.96,1271.25,1280.27,1280.27,2.728440e+09,15.10,15.19,14.20,...,1275.77,1279.36,1280.27,1277.41,1270.92,1276.66,1278.55,1263.20,1268.40,1268.88
6065,2006-08-10,1265.72,1272.55,1261.30,1271.81,1271.81,2.402190e+09,15.56,15.73,14.45,...,1268.21,1266.74,1271.81,1265.95,1271.48,1275.77,1279.36,1280.27,1277.41,1270.92
6072,2006-08-17,1295.37,1300.78,1292.71,1297.48,1297.48,2.458340e+09,12.69,12.72,12.21,...,1297.52,1302.30,1297.48,1295.43,1285.58,1268.21,1266.74,1271.81,1265.95,1271.48
6079,2006-08-24,1292.97,1297.23,1291.40,1296.06,1296.06,1.930320e+09,12.24,12.58,12.21,...,1301.78,1295.09,1296.06,1292.99,1298.82,1297.52,1302.30,1297.48,1295.43,1285.58
6086,2006-08-31,1304.25,1306.11,1302.45,1303.82,1303.82,1.974540e+09,12.23,12.33,12.07,...,1313.25,1311.01,1303.82,1305.37,1304.28,1301.78,1295.09,1296.06,1292.99,1298.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10839,2019-09-05,2960.60,2985.86,2960.60,2976.00,2976.00,3.890700e+09,16.92,17.05,15.45,...,2978.43,2978.71,2976.00,2937.78,2906.27,2926.46,2924.58,2887.94,2869.16,2878.38
10846,2019-09-12,3009.08,3020.74,3000.92,3009.57,3009.57,3.791860e+09,14.69,14.94,13.85,...,2997.96,3007.39,3009.57,3000.93,2979.39,2978.43,2978.71,2976.00,2937.78,2906.27
10853,2019-09-19,3010.36,3021.99,3003.16,3006.79,3006.79,3.251290e+09,14.66,14.66,13.31,...,2991.78,2992.07,3006.79,3006.73,3005.70,2997.96,3007.39,3009.57,3000.93,2979.39
10860,2019-09-26,2985.73,2987.28,2963.71,2977.62,2977.62,3.077240e+09,16.23,17.09,15.35,...,2976.74,2961.79,2977.62,2984.87,2966.60,2991.78,2992.07,3006.79,3006.73,3005.70


Save files

In [20]:
save_file(data_dict, "data_dict.pkl", WORK_DATA_DIR)

In [21]:
save_file(df, file_name="merged_data.csv", file_path=WORK_DATA_DIR, index=False)